In [1]:
# Section A: Data Accumulation
import json
import pandas as pd
import numpy as np
from pathlib import Path
import random

# CONFIG
DATA_ROOT = Path(r"d:\fishinggame\precompute\data\1\1001")
print(f"Data Root: {DATA_ROOT}")

# 1. Load All JSON Configs
print("Loading JSON Configs...")

# Helper to load safely
def load_json(filename):
    path = DATA_ROOT / filename
    if path.exists():
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    print(f"Warning: {filename} not found.")
    return {}

# Core Configs
fish_env_affinity_config = load_json('fish_env_affinity.json')
struct_affinity_config = load_json('struct_affinity.json')
temp_affinity_config = load_json('temp_affinity.json')
layer_affinity_config = load_json('water_layer_affinity.json')
period_affinity_config = load_json('period_affinity.json')
env_affinity_const = load_json('env_affinity_const.json')

# Global Constants
TEMP_TOLERANCE_WIDTH = env_affinity_const.get('tempToleranceWidth', 6.0)
print(f"Global Constant Loaded: TEMP_TOLERANCE_WIDTH={TEMP_TOLERANCE_WIDTH}")

# Try to load Weather Factors (Assuming it might be split or in a different file, checking for 'weather_factor.json')
# If strictly not found, we might need to mock or look into 'weather.json' if it exists
weather_factor_config = load_json('weather_factor.json') 

print("Configs Loaded.")

Data Root: d:\fishinggame\precompute\data\1\1001
Loading JSON Configs...
Global Constant Loaded: TEMP_TOLERANCE_WIDTH=6
Configs Loaded.


In [2]:
# Section B: Data Processing Helpers

# Standard Period Mapping (3-hour blocks)
PERIOD_ORDER = [
    "period0_3", "period3_6", "period6_9", "period9_12", 
    "period12_15", "period15_18", "period18_21", "period21_24"
]
PERIOD_NAME_TO_IDX = {name: i for i, name in enumerate(PERIOD_ORDER)}

def get_config_by_id(config_dict, target_id):
    """Safe lookup helper handling str/int key mismatch"""
    if target_id is None:
        return None
    return config_dict.get(str(target_id))

def build_lookup_table(source_data, group_key_name):
    """Aggregates list-based configs by group ID"""
    grouped = {}
    for item in source_data.values():
        grp_id = item.get(group_key_name)
        if grp_id is not None:
            grp_key = str(int(grp_id))
            if grp_key not in grouped:
                grouped[grp_key] = []
            grouped[grp_key].append(item)
    return grouped

# Pre-build Lookup Tables for Period
period_grouped = build_lookup_table(period_affinity_config, 'periodGroup')

def enrich_fish_data(fish_id, env_id):
    """
    Enriches a single fish entry with environmental affinity details.
    Returns a dict with all necessary columns.
    """
    env_info = get_config_by_id(fish_env_affinity_config, env_id)
    if not env_info:
        return {}
    
    row = {}
    
    # 1. Base Env IDs
    row['envId'] = env_id
    row['structId'] = env_info.get('structId')
    row['tempId'] = env_info.get('tempId')
    row['layerId'] = env_info.get('layerId')
    
    # 2. Weather/Adaptation Params
    # pressureSensitivity: float, default 0
    row['pressureSensitivity'] = env_info.get('pressureSensitivity', 0.0)
    
    # 3. Period Group
    p_grp = env_info.get('periodCoeffGroup')
    row['periodCoeffGroup'] = p_grp
    
    # 4. Struct Affinity Details (List of {structType, coeff})
    struct_info = get_config_by_id(struct_affinity_config, row['structId'])
    row['structList'] = struct_info.get('List', []) if struct_info else []
    
    # 5. Temp Affinity Details
    temp_info = get_config_by_id(temp_affinity_config, row['tempId'])
    if temp_info:
        # NOTE: Apply scaling here or in matrix build? 
        # Notebook original did it in matrix build. We will do it in matrix build to keep raw data clean.
        row['temperatureFav'] = temp_info.get('temperatureFav')
        row['tempAffectedRatio'] = temp_info.get('tempAffectedRatio')
        row['tempThreshold'] = temp_info.get('tempThreshold')
    else:
        row['temperatureFav'] = None
        row['tempAffectedRatio'] = None
    
    # 6. Layer Affinity Details
    layer_info = get_config_by_id(layer_affinity_config, row['layerId'])
    row['layerList'] = layer_info.get('List', []) if layer_info else []
    
    return row

def get_period_coeffs(period_group_id):
    """
    Returns array of size 8 (for 8 periods) with activity factors.
    Default 1.0 if not found.
    """
    if period_group_id is None:
        return np.ones(8, dtype=np.float32)
    
    grp_key = str(int(period_group_id))
    entries = period_grouped.get(grp_key, [])
    
    coeffs = np.ones(len(PERIOD_ORDER), dtype=np.float32) # Default 1.0 (Neutral)
    
    for entry in entries:
         # Entry has 'periodName' (e.g. period6_9) and 'activityFactor' (e.g. 1.2 or 0.8)
         # In data_formula: 'period_activity_factor'
         # Note: JSON keys might be camelCase. Check period_affinity.json structure from prev logs.
         # Prev logs (Cell 15 merge) showed columns: periodName, periodEntryId?
         # Need to check key names in period_affinity_config items.
         # Assuming 'periodName' and 'activityFactor' based on naming conventions.
         
         p_name = entry.get('periodName') # e.g. "period6_9"
         factor = entry.get('activityFactor', 1.0)
         
         if p_name in PERIOD_NAME_TO_IDX:
             idx = PERIOD_NAME_TO_IDX[p_name]
             coeffs[idx] = float(factor)
             
    return coeffs


In [3]:
# Section B: Matrix Construction (Refactored)

def build_dense_matrices(df_fish):
    """
    Converts enriched DataFrame into dense NumPy matrices for vectorized calc.
    
    Returns:
    - m_struct: [N_Fish, N_StructTypes]
    - m_layer:  [N_Fish, N_LayerTypes]
    - m_temp:   [N_Fish, 2] (Fav, Ratio) (Threshold ignored for now if unused or used separately)
    - m_weather:[N_Fish] (PressureSensitivity)
    - m_period: [N_Fish, N_Periods] (Activity Factors)
    """
    num_fishes = len(df_fish)
    
    # ---------------------------
    # 1. Struct Matrix (N_Fish x 32) (assuming max 32 struct types)
    # ---------------------------
    MAX_STRUCT_TYPE = 32
    m_struct = np.zeros((num_fishes, MAX_STRUCT_TYPE), dtype=np.float16)
    
    for i, rowlist in enumerate(df_fish['structList']):
        if isinstance(rowlist, list):
            for item in rowlist:
                s_type = item.get('structType')
                coeff = item.get('coeff', 0)
                if s_type is not None and 0 <= s_type < MAX_STRUCT_TYPE:
                    m_struct[i, s_type] = float(coeff)
                    
    # ---------------------------
    # 2. Layer Matrix (N_Fish x 4) (Layer 1,2,3)
    # ---------------------------
    MAX_LAYER_TYPE = 4 # 1, 2, 3
    m_layer = np.zeros((num_fishes, MAX_LAYER_TYPE), dtype=np.float16)
    
    for i, rowlist in enumerate(df_fish['layerList']):
        if isinstance(rowlist, list):
            for item in rowlist:
                l_type = item.get('layerType')
                coeff = item.get('coeff', 0)
                if l_type is not None and 0 <= l_type < MAX_LAYER_TYPE:
                    m_layer[i, l_type] = float(coeff)
                    
    # ---------------------------
    # 3. Temp Matrix (N_Fish x 2)
    # ---------------------------
    # Formula implies we need: temperatureFav, tempAffectedRatio
    # NOTE: Original notebook divided temperatureFav by 10.0 and tempAffectedRatio by 10000.0
    # verifying scaling:
    # temperatureFav usually int like 250 -> 25.0 C. 
    # tempAffectedRatio usually int like 10000 -> 1.0? Or 2000 -> 0.2?
    # Let's apply this scaling as it matches conventions seen in code.
    
    m_temp = np.zeros((num_fishes, 2), dtype=np.float32) # Higher precision for temp
    
    temp_fav_raw = df_fish['temperatureFav'].fillna(250).values # Default 25.0 C
    temp_ratio_raw = df_fish['tempAffectedRatio'].fillna(0).values 
    
    m_temp[:, 0] = temp_fav_raw / 10.0
    m_temp[:, 1] = temp_ratio_raw / 10000.0
    
    # ---------------------------
    # 4. Weather Matrix (N_Fish)
    # ---------------------------
    # Pressure Sensitivity
    m_weather = df_fish['pressureSensitivity'].fillna(0.0).values.astype(np.float16)
    
    # ---------------------------
    # 5. Period Matrix (N_Fish x 8)
    # ---------------------------
    m_period = np.zeros((num_fishes, 8), dtype=np.float16)
    
    # We iterate and compute period coeffs
    for i, p_grp in enumerate(df_fish['periodCoeffGroup']):
        coeffs = get_period_coeffs(p_grp)
        m_period[i, :] = coeffs
        
    return m_struct, m_layer, m_temp, m_weather, m_period

print("build_dense_matrices function defined.")

build_dense_matrices function defined.


In [6]:
# Section C: Data Loading & Execution

print("Loading Fish/Stock Data...")
fish_release_data = load_json('fish_release.json')
stock_release_data = load_json('stock_release.json')
# We don't merge fish_env_affinity_df fully to avoid duplicates, 
# but we read it via config loader to use in enrichment logic.

# 1. Convert to DataFrame
df_fish_release = pd.DataFrame.from_dict(fish_release_data, orient='index')
df_stock_release = pd.DataFrame.from_dict(stock_release_data, orient='index')

# 2. Rename & Merge
if not df_fish_release.empty and not df_stock_release.empty:
    df_fish_release = df_fish_release.rename(columns={'id': 'releaseId', 'name': 'fishName'})
    
    # Merge Stock -> Fish Release
    # In stock_release: "releaseId": 300010, "fishEnvId": 1010001
    df_main = df_stock_release.merge(df_fish_release, on='releaseId', how='left')
    
    print(f"Base DataFrame Shape: {df_main.shape}")
    
    # 3. Enrichment
    print("Enriching Data...")
    enriched_rows = []
    
    # Check if fishEnvId exists
    if 'fishEnvId' not in df_main.columns:
        print("Error: fishEnvId not in DataFrame!")
    else:
        for idx, row in df_main.iterrows():
            env_id = row.get('fishEnvId') 
            fish_id = row.get('mappingId') # stock_release key
            
            extra = enrich_fish_data(fish_id, env_id)
            enriched_rows.append(extra)
            
        df_extra = pd.DataFrame(enriched_rows)
        # Combine back
        df_final = pd.concat([df_main.reset_index(drop=True), df_extra.reset_index(drop=True)], axis=1)
        
        print("Enrichment Done.")
        print(f"Final DataFrame Shape: {df_final.shape}")
        
        # 4. Build Matrices
        print("Building Dense Matrices...")
        # Check if periodCoeffGroup exists
        if 'periodCoeffGroup' not in df_final.columns:
             print("Error: periodCoeffGroup missing after enrichment.")
        else:
             m_struct, m_layer, m_temp, m_weather, m_period = build_dense_matrices(df_final)
             print("Matrices Built.")

else:
    print("Error: Source Data Empty.")
    df_final = pd.DataFrame()

Loading Fish/Stock Data...
Base DataFrame Shape: (419, 15)
Enriching Data...
Enrichment Done.
Final DataFrame Shape: (419, 26)
Building Dense Matrices...
Matrices Built.


In [7]:
# Section D: Visualization & Verification

if not df_final.empty:
    print("="*40)
    print("VERIFICATION REPORT")
    print("="*40)
    
    print(f"Total Fish Stocks: {len(df_final)}")
    
    # 1. Shape Checks
    print("\n[Matrix Shapes]")
    print(f"m_struct:  {m_struct.shape} (Expect N x 32)")
    print(f"m_layer:   {m_layer.shape}  (Expect N x 4)")
    print(f"m_temp:    {m_temp.shape}  (Expect N x 2)")
    print(f"m_weather: {m_weather.shape} (Expect N,)")
    print(f"m_period:  {m_period.shape}  (Expect N x 8)")
    
    # 2. Value Checks
    print("\n[Value Statistics]")
    
    # Temp
    t_fav = m_temp[:, 0]
    t_ratio = m_temp[:, 1]
    print(f"Temp Fav:   Min={t_fav.min():.2f}, Max={t_fav.max():.2f}, Mean={t_fav.mean():.2f} (Target ~20.0)")
    print(f"Temp Ratio: Min={t_ratio.min():.4f}, Max={t_ratio.max():.4f}")
    print(f"Global Tolerance Width: {TEMP_TOLERANCE_WIDTH}")
    
    # Weather
    print(f"Pressure Sensitivity: Min={m_weather.min():.2f}, Max={m_weather.max():.2f}")
    
    # Period
    print(f"Period Coeffs Sample (First 5 rows):")
    print(m_period[:5])
    
    # 3. Logic Validation
    print("\n[Logic Validation]")
    if m_weather.sum() == 0:
        print("WARNING: m_weather is all zeros! Check pressureSensitivity loading.")
    else:
        print("PASS: m_weather contains non-zero sensitivities.")
        
    if m_period.sum() == 0:
        print("WARNING: m_period is all zeros! Check periodCoeffGroup loading.")
    else:
        print("PASS: m_period contains active factors.")
        
    # Check Temp scaling
    if t_fav.max() > 100:
        print("WARNING: Temp Fav seems unscaled (>100). Check /10.0 logic.")
    else:
        print("PASS: Temp Fav seems properly scaled.")

else:
    print("No data to verify.")

VERIFICATION REPORT
Total Fish Stocks: 419

[Matrix Shapes]
m_struct:  (419, 32) (Expect N x 32)
m_layer:   (419, 4)  (Expect N x 4)
m_temp:    (419, 2)  (Expect N x 2)
m_weather: (419,) (Expect N,)
m_period:  (419, 8)  (Expect N x 8)

[Value Statistics]
Temp Fav:   Min=7.00, Max=25.00, Mean=17.65 (Target ~20.0)
Temp Ratio: Min=0.0003, Max=0.0012
Global Tolerance Width: 6
Pressure Sensitivity: Min=0.00, Max=1.20
Period Coeffs Sample (First 5 rows):
[[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]

[Logic Validation]
PASS: m_weather contains non-zero sensitivities.
PASS: m_period contains active factors.
PASS: Temp Fav seems properly scaled.


In [8]:
# Section E: Simulation Demo (Calculation Loop Verification)

print("="*40)
print("SIMULATION DEMO")
print("="*40)

# 1. Mock Input Data (Small 10x10x10 Grid)
X, Y, Z = 10, 10, 10
N_Fish = m_struct.shape[0]

# Mock Bitmask (Random features)
struct_bitmask_map = np.random.randint(0, 4096, (X, Y, Z), dtype=np.int32) 
# Mock Depth Map (0 to 1)
normalized_depth_map = np.linspace(0, 1, Y).reshape(1, Y, 1)
normalized_depth_map = np.tile(normalized_depth_map, (X, 1, Z))

# 2. Mock Environment State
current_pressure_influence = 0.8 # Hypo: from WeatherFactor['pressure_influence'] based on current pressure
current_period_idx = 2 # e.g. period6_9 (Index 2 in PERIOD_ORDER)
current_surface_t = 22.0
current_bottom_t = 15.0

print(f"Scenario: SurfaceT={current_surface_t}, BottomT={current_bottom_t}, PressureInf={current_pressure_influence}, PeriodIdx={current_period_idx}")

# 3. Calculation Logic

# A. Struct Affinity
# Simplified for demo: just check one bit (e.g. Bit 1)
aff_struct = np.ones((X, Y, Z, N_Fish), dtype=np.float32) * 0.1 # Base 0.1
# (Real logic iterates bits as in original notebook)

# B. Temp Affinity
# T_Map
t_map = current_surface_t + (current_bottom_t - current_surface_t) * normalized_depth_map # [X,Y,Z]
t_map_exp = t_map[..., np.newaxis] # [X,Y,Z,1]
t_fav = m_temp[:, 0]
t_ratio = m_temp[:, 1]
# Gaussian
diff_sq = (t_map_exp - t_fav) ** 2
denom = TEMP_TOLERANCE_WIDTH * (t_ratio ** 2)
denom[denom < 1e-9] = 1e-9
aff_temp = np.exp(- diff_sq / denom)

# C. Weather Affinity (Pressure)
# Formula: k_pressure = pressure_influence ^ pressureSensitivity
# m_weather is pressureSensitivity [N_Fish]
# Broadcast to [X,Y,Z, N_Fish]
p_sens = m_weather
aff_weather_vec = np.power(current_pressure_influence, p_sens) # [N_Fish]
aff_weather = np.tile(aff_weather_vec, (X, Y, Z, 1))

# D. Period Affinity
# m_period [N_Fish, 8]
factor_vec = m_period[:, current_period_idx] # [N_Fish]
aff_period = np.tile(factor_vec, (X, Y, Z, 1))

# E. Final Synthesis
aff_total = aff_struct * aff_temp * aff_weather * aff_period

print("\n[Simulation Results]")
print(f"Aff Total Shape: {aff_total.shape}")
print(f"Max Affinity: {aff_total.max()}")
print(f"Mean Affinity: {aff_total.mean()}")

# Validation checks
if aff_total.max() > 0:
    print("PASS: Simulation produced non-zero weights.")
else:
    print("WARNING: All weights are zero (could be due to narrow temp tolerance or mock struct).")


SIMULATION DEMO
Scenario: SurfaceT=22.0, BottomT=15.0, PressureInf=0.8, PeriodIdx=2

[Simulation Results]
Aff Total Shape: (10, 10, 10, 419)
Max Affinity: 0.10000000149011612
Mean Affinity: 0.0007868553406331726
PASS: Simulation produced non-zero weights.


New Notebook Created by Jupyter MCP Server